In [43]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib
import re

In [44]:
# Load dataset
df = pd.read_csv('spam.csv')

# Remove null values
df = df.dropna()

In [45]:
df.head()

,comment,label
0,Subject: enron methanol ; meter # : 988291\r\n...,0
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,"Subject: photoshop , windows , office . cheap ...",1
4,Subject: re : indian springs\r\nthis deal is t...,0


In [46]:
# Function to clean comments
def clean_comment(comment):
    # Remove URLs
    comment = re.sub(r'http\S+|www\S+|https\S+', '', comment, flags=re.MULTILINE)
    # Remove special characters and numbers
    comment = re.sub(r'\@\w+|\#','', comment)
    comment = re.sub(r'[^A-Za-z\s]', '', comment)
    # Convert to lowercase
    comment = comment.lower()
    return comment

# Clean the comments
df['comment'] = df['comment'].apply(clean_comment)

In [47]:
df.head()

,comment,label
0,subject enron methanol meter \r\nthis is a ...,0
1,subject hpl nom for january \r\n see attache...,0
2,subject neon retreat\r\nho ho ho we re aroun...,0
3,subject photoshop windows office cheap mai...,1
4,subject re indian springs\r\nthis deal is to ...,0


In [48]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['comment'], df['label'], test_size=0.2, random_state=42)

# Vectorize the comments
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [49]:
# Initialize and train the Random Forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_vect, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_vect)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.97


In [50]:
# Save the model and vectorizer
joblib.dump(model, 'spam_classifier_model.joblib')
joblib.dump(vectorizer, 'vectorizer.joblib')

['vectorizer.joblib']

In [53]:
# Load the saved model and vectorizer for single prediction
loaded_model = joblib.load('spam_classifier_model.joblib')
loaded_vectorizer = joblib.load('vectorizer.joblib')

# Single prediction
def predict_comment(comment):
    cleaned_comment = clean_comment(comment)
    vect_comment = loaded_vectorizer.transform([cleaned_comment])
    prediction = loaded_model.predict(vect_comment)
    return 'spam' if prediction[0] == 1 else 'ham'

sample_comment = "Congratulations! You've won a $1000 Walmart gift card. Click here to claim now."
# sample_comment = "How are you man, please call me once you are back."
prediction = predict_comment(sample_comment)
print(f'The sample comment is predicted to be: {prediction}')

The sample comment is predicted to be: spam
